In [160]:
from datetime import datetime, timedelta
import os, argparse, sys
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd
from glob import glob
from datetime import datetime, timedelta
from dateutil.parser import parse as parse_to_datetime
import imageio.v2 as imageio
import openpyxl
import sklearn


In [161]:
######################## PROFILE PLOT ####################################   
## User Input from excel config file

##
# Raise Exception indicating problem with configuration sheet
## 

if len(config.columns) == 0:
    raise Exception("No Data found in configuration sheet ")

#reading in user input from excel config excel file
config_path = "makeprofileplot_config.xlsx" #Location of excel user file
config = pd.read_excel(config_path, skiprows=2, engine='openpyxl', usecols='A:C', index_col=0, nrows=25) # Need to download openpyxl package to import xlsx
skiprows = config.at['Skip Rows', config.columns[0]]
x_label = config.at['X Label', config.columns[0]]
x_min = config.at['X Axis (min, max)', config.columns[0]]
x_max = config.at['X Axis (min, max)', config.columns[1]]
y_label = config.at['Y Label', config.columns[0]]
y_min = config.at['Y Axis (min, max)', config.columns[0]]
y_max = config.at['Y Axis (min, max)', config.columns[1]]
start_day = config.at['Julian Start Day', config.columns[0]]
obsdatapath = config.at['File', config.columns[0]]
obs_day_column_name = config.at['Date Column', config.columns[0]]
obs_variable = config.at['Variable Name', config.columns[0]]
obs_variable_units = config.at['Variable Units', config.columns[0]]
obs_variable_column_name = config.at['Variable Column Name', config.columns[0]]
obs_depth_column_name = config.at['Depth Column Name', config.columns[0]] 
obs_result_column_name = config.at['Result Column Name', config.columns[0]]
obs_na_values = config.at['NA Values', config.columns[0]]
figure_title = config.at['Figure Title', config.columns[0]]
obs_site = config.at['Site Name', config.columns[0]]
obs_site_column_name = config.at['Site Column Name', config.columns[0]]
obs_units_column_name = config.at['Units Column Name', config.columns[0]]
obs_units = config.at['Variable Units', config.columns[0]]
obs_units2 = config.at['Variable Units(2)', config.columns[0]]
# NOTE: append "_column_name" to variables that store labels (names of columns)
modpath = config.at['File', config.columns[1]]
mod_day_column_name = config.at['Date Column', config.columns[1]]
mod_variable = config.at['Variable Name', config.columns[1]]
mod_variable_column_name = config.at['Variable Column Name', config.columns[1]]
mod_depth_column_name = config.at['Depth Column Name', config.columns[1]] 
mod_result_column_name = config.at['Result Column Name', config.columns[1]]
mod_na_values = config.at['NA Values', config.columns[1]]
mod_variable_units = config.at['Variable Units', config.columns[1]]

profileplotfolder = config.at['Profile Plots Folder', config.columns[0]] 
statsfolder = config.at['Statistic Output Folder', config.columns[0]]

In [162]:
# loading in model and observed data

print('loading model "%s"' % modpath)
print('loading observed "%s"' % obsdatapath)

#read in profile model outputs
moddata = pd.read_csv(modpath, na_values=mod_na_values)

#observed data
obsdata = pd.read_csv(obsdatapath, na_values= obs_na_values)

loading model "data\Model Files\spr_wb1_example.csv"
loading observed "data\Model Files\SampleData.csv"


In [163]:
#Conditional Statements - Create tables for observed and modeled data
#### This requires data to be in a specific format ####
#Create model data table, round down days
profobs = obsdata[(obsdata[obs_site_column_name] == obs_site)].copy()
profobs = profobs[(profobs[obs_variable_column_name] == obs_variable)]
profobs = profobs[(profobs[obs_units_column_name] == obs_units) | (profobs[obs_units_column_name] == obs_units2)]
profobs = profobs[[obs_day_column_name, obs_depth_column_name, obs_result_column_name]] #making a table with day, depth, and results
profobs[obs_day_column_name] = profobs[obs_day_column_name].apply(np.floor) #round down
profobs = profobs.dropna()

#Create observed data table, round down days
profmod = moddata[(moddata[mod_variable_column_name] == mod_variable)].copy() 
profmod = profmod[[mod_day_column_name, mod_depth_column_name, mod_result_column_name]]
profmod[mod_day_column_name] = profmod[mod_day_column_name].apply(np.floor)
profmod = profmod.dropna()

#use this if you want to see the data
#print('Observed data:')
#print(profobs)
#print('Model data:')
#print(profmod)

In [164]:
#create a model days and observed days into a set
modeldays = set(profmod[mod_day_column_name])
observeddays = set(profobs[obs_day_column_name])
days =  modeldays.intersection(observeddays) #find days within the data set that match

In [165]:
#Create Index of Days in Model Dataset - In the future we may need to check that observed and modeled days match earlier in code
#mod_ind = profmod_complete[mod_day].unique() # prof mod complete not defined
#mod_ind #Index of Julian Days for model dataset. We are assuming the model days match the observed days.

# renaming variables so they specify that they are interpolated!
interpolated_df_day_column_name = mod_day_column_name
interpolated_df_depth_column_name = 'Depth' #mod_depth_column_name
interpolated_df_mod_result_column_name = mod_result_column_name
interpolated_df_obs_result_column_name = obs_result_column_name
interpolated_columns = [
    interpolated_df_day_column_name, 
    interpolated_df_depth_column_name, 
    interpolated_df_mod_result_column_name, 
    interpolated_df_obs_result_column_name,
]


interpolated_df = pd.DataFrame(columns=interpolated_columns) #creating an empty data frame to put all of the interpolated values in

#Interpolate - each days values are interpolated using this loop

for i in days: # changed this to modeldays instead of mod_ind
    profmod_i = profmod[(profmod[mod_day_column_name] == i)]
    profobs_i = profobs[(profobs[obs_day_column_name] == i)]
    
    mod_depths = profmod_i[mod_depth_column_name]
    mod_results = profmod_i[mod_result_column_name]
    
    obs_depths = profobs_i[obs_depth_column_name]
    obs_results = profobs_i[obs_result_column_name]

    # if there are no observed depths for day "i" np.interp will crash below. 
    # so, if there are none, skip this day (via "continue")
    # if len(obs_depths) == 0:
    #     print('no observed depths for day {}'.format(i))
    #     continue
    
    interp_mod_results = list(np.interp(obs_depths, mod_depths, mod_results))
    interp_mod_days = [i] * len(obs_depths)
    interpolated_df = pd.concat([
        interpolated_df, 
        pd.DataFrame(zip(interp_mod_days, obs_depths, interp_mod_results, obs_results), columns=interpolated_columns)
    ])

# new data frame will have day, depth, interpolated model data, observed data

In [166]:
#Function to Caluclate Statistic Values
from sklearn.metrics import mean_absolute_error, mean_squared_error

stats_columns = [
    'DAY',
    'MAE', 
    'RMSE', 
    'ME',
    'MODEL ST.DEV',
    'PBIAS',
    'MOD_MEAN',
    'OBS_MEAN',
    'Data Points'
]
def make_empty_statsdf():
    return pd.DataFrame(columns=stats_columns)

def concat_statsdf(statsdf, df, day):
    y_true = df[interpolated_df_obs_result_column_name].to_numpy()
    y_pred = df[interpolated_df_mod_result_column_name].to_numpy()
    MOD_MEAN = y_pred.mean()
    OBS_MEAN = y_true.mean()
    num_data_points = len(y_true)
    RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    ME = np.sum(y_pred - y_true) / num_data_points
    MAE = np.sum(np.absolute(y_pred - y_true))/ num_data_points
    MOD_ST_DEV = y_pred.std()
    PBIAS = 100 * np.sum(y_true - y_pred) / np.sum(y_true)
    return pd.concat([
        statsdf,
        pd.DataFrame([[day, MAE, RMSE, ME,MOD_ST_DEV, PBIAS, MOD_MEAN, OBS_MEAN, num_data_points]], columns=stats_columns)
    ])


In [167]:
# iterate over the interpolated_df we created above
#creating the profile plots

statsdf = make_empty_statsdf()
images = [] #Create empty list for profile plot file names so that .gif and .avi show days in order

loop_count = 0

for i in sorted(days):
    interp_i = interpolated_df[(interpolated_df[interpolated_df_day_column_name] == i)]
    mod_i = profmod[(profmod[mod_day_column_name]==i)]
    depths = list(interp_i[interpolated_df_depth_column_name])
    mod_depths = list(mod_i[mod_depth_column_name])
    x_mod = list(mod_i[mod_result_column_name])
    x_obs = list(interp_i[interpolated_df_obs_result_column_name])
    date = (start_day) + timedelta(days=(i-1))
    fig, ax = plt.subplots()
    ax.plot(x_mod, mod_depths, marker = '', linestyle ='-', label = 'Model')
    ax.plot(x_obs, depths, marker ='*', linestyle = 'None', color ='g', label = 'Observed')
    plt.title(f"{figure_title} {date.strftime('%B %d %Y')}" )
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(loc = 'lower right')
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.invert_yaxis()
    ax.axis([x_min, x_max, y_max, y_min])
    ax.text( 1, 2 , f"Julian Day: {i}")
    plotname = f'profmod_{i}.jpg'
    images.append(os.path.join(profileplotfolder, plotname))
    fig.savefig(os.path.join(profileplotfolder, plotname))
    fig.clf()
    
    statsdf = concat_statsdf(statsdf, interp_i, i).sort_values('DAY') # calling the statistics function in this loop to calculate for everyday
    
    loop_count = loop_count + 1

print(f'loop processed {loop_count} days')
    
statsdf = concat_statsdf(statsdf, interpolated_df, 'AVG') #average statistics values
statsdf.to_csv(os.path.join(statsfolder, 'Statistics.csv'))

C:\Users\joshg\AppData\Local\Temp\ipykernel_25404\3155941429.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


loop processed 23 days


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [168]:
# Create Gif of profile plots (Cui Yong, 2020)
from pathlib import Path
image_path = Path(profileplotfolder)
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
len(image_list)
imageio.mimwrite(os.path.join(profileplotfolder, 'profileplots.gif'), image_list , fps =3)
file_name

'plots\\modelprofileplots\\profmod_721.0.jpg'

In [169]:
#Create video .avi
import cv2
import os

image_folder = Path(profileplotfolder)
video_name = 'Profiles.avi'
frame = cv2.imread(os.path.join(images[0]))
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()
